In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Bidirectional, GRU
from keras.layers import TimeDistributed
from keras.optimizers import Adam

I:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


input: [batch_size, max_words(time step), word_vec_dim]

In [7]:
nr_hidden = 16
dropout = 0.5
nr_class = 5

model = Sequential()
model.add(TimeDistributed(Dense(nr_hidden, use_bias=False), input_shape=(None, 300)))
model.add(Bidirectional(GRU(nr_hidden,
                            recurrent_dropout=dropout,
                            dropout=dropout)))
model.add(Dense(nr_class, activation='sigmoid'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

Tensorboard callback

In [8]:
board_callback = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, batch_size=32,
                                             write_graph=True, write_grads=True,
                                             write_images=True, embeddings_freq=0, embeddings_layer_names=True,
                                             embeddings_metadata=True)


Instructions for updating:
Use the retry module or similar alternatives.


Save model callback

In [11]:
checkpoint_path = "checkpoints/"

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_path + "epoch_{epoch:02d}-val_los_{val_loss:.2f}.hdf5", 
                                                            monitor='val_loss', 
                                                            verbose=0, 
                                                            save_best_only=True, 
                                                            save_weights_only=False, 
                                                            mode='auto', 
                                                            period=1)

Plot loss and accuracy

class PlotLearning(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1

    def plot(self, logger):
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="accuracy")
        ax2.plot(self.x, self.val_acc, label="validation accuracy")
        ax2.legend()
        
        plt.savefig(logger.get_log_path("loss", ".png"))

![loss_accuracy](loss.png)

In [18]:
def fit(x_train, y_train, epochs):
    plot_learing = PlotLearning()

    model.fit(x_train, y_train, validation_data=(self.ds.x_val, self.ds.y_val),
              epochs=epochs, batch_size=500, 
              callbacks=[board_callback,model_checkpoint_callback, plot], verbose=self.verbose)

    plot_learing.plot()